In [1]:
import findspark
findspark.init()

import pyspark

from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder.getOrCreate()

21/08/21 10:25:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
spark

In [3]:
%load_ext sparksql_magic

In [4]:
%%sparksql
SHOW TABLES

database,tableName,isTemporary


In [5]:
%alias_magic --cell sql sparksql

Created `%%sql` as an alias for `%%sparksql`.


In [6]:
%%sql
DROP TABLE student

AnalysisException: Table or view not found for 'DROP TABLE': student; line 1 pos 0;
'DropTable false, false
+- 'UnresolvedTableOrView [student], DROP TABLE, true


In [9]:
%%sql
CREATE TABLE student (
    id INT, 
    name STRING, 
    age INT,
    books ARRAY<STRUCT<`title`:STRING, `chapters`:ARRAY<STRUCT<`paragraph`:STRING>>>>,
    struct_col STRUCT<`address`:STRUCT<`streetName`:STRING, `streetNumber`:BIGINT>>,
    map_col MAP<STRING, MAP<STRING, STRUCT<`start`:BIGINT,`end`:BIGINT>>>

    ) USING PARQUET
    COMMENT 'this is a comment'
    TBLPROPERTIES ('foo'='bar');

In [10]:
%%sql
ALTER TABLE student ALTER COLUMN name COMMENT "new comment";

In [11]:
%%sql
-- comments go here
SHOW TABLES

database,tableName,isTemporary
default,student,False


In [12]:
%%sql
-- comments go here
DESCRIBE TABLE student

col_name,data_type,comment
id,int,null
name,string,new comment
age,int,null
books,"array<struct<title:string,chapters:array<struct<paragraph:string>>>>",null
struct_col,"struct<address:struct<streetName:string,streetNumber:bigint>>",null
map_col,"map<string,map<string,struct<start:bigint,end:bigint>>>",null


In [13]:
from pyspark.sql.types import *

def getTypeName(t):
    if type(t) == LongType:
        return 'long'
    if type(t) == IntegerType:
        return 'integer'
    if type(t) == StringType:
        return 'string'
    if type(t) == ArrayType:
        return 'array'
    if type(t) == StructType:
        return 'struct'
    if type(t) == MapType:
        return 'map'

def getPath(path, name):
    if len(path) > 0:
        return f'{path}.{name}'
    return name

def getChildren(field, path, fields):
    if type(field) == StructField:
        getChildren(field.dataType, getPath(path, field.name), fields)
    elif type(field) == MapType:
        #print('map ' + path + " " + getTypeName(field.valueType))
        getChildren(field.valueType, getPath(path, 'key'), fields)
    elif type(field) == ArrayType:
        getChildren(field.elementType, path, fields)
    elif type(field) == StructType:
        for name in field.fieldNames():
            child = field[name]
            fields.append({
                'columnName': getPath(path, name),
                'metadata': child.metadata, 
                'type': getTypeName(child.dataType),
                'description': getTypeName(child.dataType)
            })
            getChildren(child, path, fields)

df = spark.table('student')
fields = []
getChildren(df.schema, '', fields)
for f in fields:
    print(f)

{'columnName': 'id', 'metadata': {}, 'type': 'integer', 'description': 'integer'}
{'columnName': 'name', 'metadata': {'comment': 'new comment'}, 'type': 'string', 'description': 'string'}
{'columnName': 'age', 'metadata': {}, 'type': 'integer', 'description': 'integer'}
{'columnName': 'books', 'metadata': {}, 'type': 'array', 'description': 'array'}
{'columnName': 'books.title', 'metadata': {}, 'type': 'string', 'description': 'string'}
{'columnName': 'books.chapters', 'metadata': {}, 'type': 'array', 'description': 'array'}
{'columnName': 'books.chapters.paragraph', 'metadata': {}, 'type': 'string', 'description': 'string'}
{'columnName': 'struct_col', 'metadata': {}, 'type': 'struct', 'description': 'struct'}
{'columnName': 'struct_col.address', 'metadata': {}, 'type': 'struct', 'description': 'struct'}
{'columnName': 'struct_col.address.streetName', 'metadata': {}, 'type': 'string', 'description': 'string'}
{'columnName': 'struct_col.address.streetNumber', 'metadata': {}, 'type': 'l

In [14]:
def getColumns(name):
    fields = []
    getChildren(spark.table(name).schema, '', fields)
    return fields

def getTables(database):
    rows = spark.sql(f'SHOW TABLES IN {database}').collect()
    return list(map(lambda r: {
        "tableName": r.tableName,
        "columns": getColumns(r.tableName), 
        "database": None
    }, rows))

def getDescription(name):
    rows = spark.sql(f'DESCRIBE FUNCTION EXTENDED {name}').collect()
    textLines = list(map(lambda r: r.function_desc, rows))
    return "\n".join(textLines)

def getFunctions():
    rows = spark.sql('SHOW FUNCTIONS').collect()
    return list(map(lambda f: {
        "name": f.function, 
        "description": getDescription(f.function)
    }, rows))

def getConfig():
    return {
        "tables": getTables('default'), 
        "functions": getFunctions()
    }
    
import json
outputFileName = './all.json'
with open(outputFileName, 'w') as fout:
    json.dump(getConfig(), fout, sort_keys=True, indent=2)